In [1]:
import logging

import requests

from yamm.constructs.road import Road
from yamm.matchers.matcher_interface import *
from yamm.utils.crs import LATLON_CRS
from yamm.utils.url import multiurljoin

log = logging.getLogger(__name__)

DEFAULT_OSRM_ADDRESS = "http://router.project-osrm.org"


def parse_osrm_json(j: dict, trace: Trace) -> List[Match]:
    """
    parse the json response from the osrm match service

    we're looking for the osm node ids which should be in the form:

    { 'matchings':
        [{ 'legs':
            [{ 'annotations':
                {'nodes': [node_id, node_id] }
            }]
        }]
    }

    :param j: the json object
    :return:
    """
    matchings = j.get("matchings")
    if not matchings:
        raise ValueError("could not find any link matchings in response")

    legs = matchings[0].get("legs")
    if not legs:
        raise ValueError("could not find any link legs in response")

    def _parse_leg(d: dict, i: int) -> Match:
        annotation = d.get("annotation")
        if not annotation:
            raise ValueError("leg has no annotation information")
        nodes = annotation.get("nodes")
        if not nodes:
            raise ValueError("leg has no osm node information")
        link_id = f"({nodes[0]},{nodes[1]})"

        #TODO: we need to get geometry and distance info from OSRM if available
        road = Road(road_id=link_id, geom=None)
        #TODO- try/except the geometry from the OSRM resp
        
        match = Match(road=road, coordinate=trace.coords[i], distance=None)
        


/Users/grahamwaters/opt/anaconda3/envs/yamm/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
